In [ ]:
!pip install python-dotenv

In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from openai import OpenAI
# client = OpenAI(api_key = os.environ['OPENAI_KEY'])
client = OpenAI()
def ask_llm(prompt, model = 'gpt-5-nano', temp=1):
    response = client.chat.completions.create(
        model=model,    
        temperature=temp,
        messages=[{
            'role':'user',
            'content' : prompt
        }] 
    )       
    return response.choices[0].message.content   

In [ ]:
# 1. zero-shot Prompting
# 예시없이 지시사항만 던지는것 - LLM의 기본기능
prompt = "이 문장의 감정을 분류해: '오늘 점심 메뉴가 품절이라 너무 슬퍼.'"
print(ask_llm(prompt,temp=1))

In [ ]:
# 2. few-shot Prompting
# 이렇게 하는거야 라고 예시(Shot)를 몇개 보여줘서 성능을 높이는 기술
prompt = """
단어를 이모지로 바꿔줘
사과 -> 🍎
자동차 -> 🚗
고양이 -> 🐱
비행기->
"""
print(ask_llm(prompt,temp=1))

In [ ]:
# 3. Chain-of-Thought Prompting  Cot(생각의 사슬)
prompt = """
질문 : 5개의 사과중 2개를 먹고 3개를 더 샀어. 몇개 남았지
사과의 단가는 100원
지급한 금액 : 1000원
총 남은 사과의 개수를 세고 그리고 사과를 구입할때 드는 비용을 
계산해서 거스름돈을 계산해줘
계산은 먹은 사과와 남은 사과를 모두 포함한 금액
마지막 출력은 전체 로직을 점검해서 오류가 있는지 확인하고 결과알려줘
"""
print(ask_llm(prompt,temp=1))

In [ ]:
# 4. self-Consistency(자기 일관성)
# 한번만 묻지 않고 여러번(예 : 3번) 물어본 뒤 가장 많이 나온 답을 채택함
question = '철수는 학교까지 10분 걸려, 왕복은 몇분 걸릴까?'
answer = []
for _ in range(3):
    answer.append(ask_llm(question))
print(f'수집된 답변들: ', answer)
from collections import Counter
counter = Counter(answer)
counter.most_common(1)

In [ ]:
# 5. Generate Knowlege Prompting(지식생성)
# 바로 답하지 말고 관련된 지식을 먼저 생성한뒤에 그 지식을 바탕으로 답하게 됨
# 1 지식생성
knowledge = ask_llm('골프라는 스포츠에 대해 사실적인 지식 3가지만 나열해줘')
print(f'[지식] : {knowledge}')
# 2 단계 : 지식을 활용해 답변
prompt = f"""
다음 지식을 참고해서 '골프에서 홀인원이 왜 어려운지' 설명해줘.
모든 답변은 한글로 작성
[지식] : {knowledge}
"""
print(ask_llm(prompt))

In [ ]:
prompt = '골프에서 홀인원이 왜 어려운지 설명해줘'
print(ask_llm(prompt))

In [ ]:
# 6. Prompt Chaining(프롬프트 체이닝)
# 복잡한 일을 한번에 시키지 않고 A작업의 결과를 B작업의 입력으로 넘겨주는 파이프라인
# Step 1 : 주체 추출
text = '이메일: 안녕하세요, 이번 주 금요일 회의는 2시로 변경되었습니다.'
topic = ask_llm(f'다음 텍스트에서 핵심 주제만 단어로 뽑아줘 출력은 한글로:{text}')

# step2 : 답장 작성
reply = ask_llm(f"'{topic}'에 대해 '알겠습니다'라는 정중한 답장 메일을 써줘")
print(reply)

In [ ]:
# 8 Retrieval Augmented Generation(RAG 검색 증강 생성)
# 이론 : LLM이 모르는 외부 데이터(회사문서등)을 찾아서 (Retrieval)프롬프트에 넣어주고 답하게 함

# 가상의 검색된 문서
retrieved_doc = "문서내용: 우리 회사의 재택근무는 매주 수요일만 가능하다"

prompt = f'''
아래[참조문서]를 기반으로 답변해, 문서에 없으면 모른다고 해.
[참조문서] : {retrieved_doc}
질문 : 재택근무는 언제 할 수 있어?
'''
print(ask_llm(prompt))

In [ ]:
# 9 Automatic Reasoning and Tool-use 자동 추론 및 도구 사용
# LLM이 스스로 계산기나 검색엔진 같은 도구가 필요한지 판단하고 호출형식을 뱉어내는 것
prompt = '''
계산이 필요하면 [CALC: 수식] 이라고 출력해.
질문 : 3452 * 192는 뭐야?
'''
response = ask_llm(prompt)
print(response)


In [ ]:
prompt = """
너는 자동 도구 선택 시스템이야.
다음과 같은 도구를 사용할 수 있어:

1.계산기 -> [CALC : 수식]
2.날씨 조회 ->[WEATHER : 도시명]
3.일반질문 - >직접입력

규칙:
- 계산이 필요하면 CALC: ... 출력
- 날씨 정보가 필요하면 WEATHER: 도시명 출력 도시명은 영문, json구조를 파싱해서 날씨정보를 읽어서
사용자에게 친화적인 답변으로 변경
- 그 외 는 일반적인 답변

질문: 3458 * 256의 결과와 
서울의 내일 날씨는 어때?
"""

def find_weather(CITY= 'Seoul'):    
    API_KEY = os.environ['OPEN_WEATHER_KEY']
    lat = 37.25
    lon = 126.45
    print(API_KEY)
    CITY = 'Seoul'
    url = f'https://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}'
    import requests
    response = requests.get(url)
    return response.text

import re
def process_response(text):
    # 계산기
    calc = re.findall(r'CALC:\s*(.*)', text)
    if calc:
        expr = calc[0]  
        print(f'계산기 expr = {expr}')      
        return eval(expr)
    # 날씨
    weather = re.findall(r'WEATHER:\s*(.*)', text)
    if weather:
        city = weather[0]
        return find_weather(city)

response = ask_llm(prompt)
print(f'llm 출력 결과 : {response}')

# for res in response.split('\n'):
#     process_response(res)

In [ ]:
pattern = r"\[(\w+)\s*:\s*([^\]]+)\]"
matches = re.findall(pattern, response)
eval(matches[0][1])

In [ ]:
(matches[1][1])

In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()
API_KEY = 'fdf46d9b90a6dce13b4e54db8621e743'
lat = 37.25
lon = 126.45
print(API_KEY)
CITY = 'Seoul'
url = f'https://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}'
import requests
response = requests.get(url)
response.text

In [ ]:
# 10 Automatic Prompt Enginerr (APE)
# 사람이 프롬프트는 짜는게 아니라 LLM에게 좋은 프롬프트를 짜줘 라고 시키는 것
task = '고객의 리뷰에서 감정을 분석하는 작업'
prompt = f'''
나는 '{task}'을 하려고 해.
이 작업을 수행하기에 가장 완벽한 프롬프트 지시문을 작성해줘 바로 사용할수 있도록 간결하게 작성
'''
best_prompt = ask_llm(prompt)
print(best_prompt)

In [ ]:
# 11. Active-Prompt
# LLM이 답변하기 애매하거나 불확실한 문제를 찾아내서 사람에게 이것좀 가르쳐 주세요(예시추가) 라고 요청하는 방식
# LLM에게 문제를 풀게하고 '확신도(Confidence score)를 묻는다 낮으면 그 문제를 few-shot에 예제로 추가

# 프롬프트 생성
import json
def build_prompt(task:str,examples:list,query:str) -> str:
    prompt = f'작업: {task}\n\n'
    for ex in examples:
        prompt += f"예시 입력 : {ex['input']}\n예시출력:{json.dumps({'answer':ex['answer'],
                                        'confidence':ex.get('confidence',0.9)})}\n\n"
    prompt += f'입력:{query}\njson 형식으로 answer ,confidence 반환'
    return prompt

def call_llm(prompt:str, model='gpt-5-nano') -> dict:
    resp = client.chat.completions.create(
        model=model,
        messages= [{'role':'user', 'content':prompt}]
    )
    text = resp.choices[0].message.content
    try:
        return json.loads(text)
    except:
        return {'answer':text, 'confidence':0.0}
# Active-prompt 루프
def active_prompt(task:str, query:str, examples=None):
    if examples is None:
        examples = []
    for i in range(4):
        prompt = build_prompt(task, examples, query)
        result = call_llm(prompt)
        print(f"\nIteration : {i+1}: answer={result['answer']}, \
               confidence={result.get('confidence',0)}")
        if result.get('confidence',0) >= 0.75:
            break
        else:
            # 낮은 confidence -> few shot 예제로 추가
            examples.append({"input":query,'answer':result['answer'], 
                             "confidence":result.get('confidence',0)})
    return result
# 실행 : 명확한 문제
task1= '숫자가 소수인지 판단'
query1 = '97은 소수인가요?'
res1 = active_prompt(task1,query1)
print(res1)
# 실행 : 애매한 문제
task2= '주어진 문장을 더 정중하고 간결하게 바꿔주세요 너무 직설적이거나 모호하면 안됨'
query2 = '이 프로젝트는 결과가 별로인거 같아요, 좀더 괜찮게 고쳐주세요'
res2 = active_prompt(task2,query2)
print(res2)


In [ ]:
# 12 Directional Stimulus Prompting(방향성 자극)
# 모델에게 구체적이 지시를 내리기전에 힌트(키워드)를 제공해서 답변의 방향을 유도
article = '''
로봇 산업이 '피지컬 인공지능(AI)'과 만나 새로운 확장기를 맞아가고 있다. 피지컬 AI란 물리적 세계를 인식·이해하고 직접 상호작용하는 행동형 AI를 의미한다. 기계의 '뇌'가 더 똑똑해질수록 제조·국방 등 핵심 분야에서 로봇이 유의미하게 쓰일 수 있다. 시장조사 업체 슈타티스타는 전 세계 피지컬 AI시장 규모가 올해 225억달러에서 2030년 643억달러로 성장할 것으로 전망했다.

로봇은 피지컬 AI의 꽃으로 불린다. 미국과 중국 등 패권국은 이미 피지컬 AI와 결합한 로봇 산업을 국가적 '전략자산'으로 육성하고 있다. 특히 주요국들은 고령화·저출생에 따른 일손 부족, 인건비 상승의 흐름 속에서 제조업을 혁신할 키워드로 로봇에 주목하고 있다. 제조 강국인 한국이 주도권을 확보할 수 있는 분야로도 평가된다.

이러한 가운데 로봇 분야 세계 최고 권위 학회이자 로봇 연구자 수만 명이 활동하는 커뮤니티인 전기전자공학자협회(IEEE) 산하 국제로봇·자동화학회(RAS)를 한국인이 이끌게 돼 주목된다. RAS 집행부는 지난달 학술대회를 열고 조규진 서울대 공대 교수를 차기 회장으로 선출했다. RAS 회장은 글로벌 로봇 분야의 학술·산업 어젠다를 조율하는 핵심 포지션이다.

조 교수는 내년 1월부터 2027년 12월까지 회장 당선인으로 활동하며 2028년 1월부터 2029년 12월까지 RAS를 이끌게 된다. 로봇 업계와 학계에서는 한국 로봇 연구 커뮤니티의 국제 위상과 리더십이 한층 강화될 것이라는 기대가 나온다.

조 교수는 생체모사 로봇, 소프트 로봇, 웨어러블 로봇 분야의 세계적인 석학이다. 그를 만나 로봇 학계 최신 트렌드와 RAS 회장 선출의 의미, 한국이 나아가야 할 방향 등을 들어봤다. 다음은 일문일답.
'''

In [ ]:
prompt = f"""
기사:{article}
힌트(키워드) : 소프트웨어, AI
위 힌트를 중심으로 기사를 요약해줘
"""
print(ask_llm(prompt))

In [ ]:
# 13 Program-Aided Language Models(PAL)
# 수학문제나 날짜 계산은 말로 풀지 말고 파이썬 코드를 짜서 해결하도록 유도
# LLM은 코딩을 더 잘함
prompt = '''
질문 : 2025년 11월 24일에서 90일 후는 무슨요일이야?
이 문제를 해결하는 python코드를 작성하고 해당 코드를 이용해서 알려줘
'''
print(ask_llm(prompt))

In [ ]:
# 14 ReAct(Reason + Act)
# 생각(Reason)하고 -> 행동(Act)하고 -> 관찰(Observation)하는 과정을 반복하면서 문제를 해결
# 에이전트(Agent)의 기초
prompt = '''
질문 : 손흥민 나이에 10살을 더하면?
다음 형식에 따라서 진행
Thought: 손흥민의 생년월일을 검색한다
Action : Search[손흥민의 생일]
Observation : (검색결과 기디림)
'''
print(ask_llm(prompt))


In [ ]:
# 15 Reflexion(리플렉션 / 반성)
# 모델이 틀린답을 냈을때 왜 틀렸는지 반성(Reflect)하고 다시 답을 출력 하는 과정
wrong_answer = "파이썬 리스트 추가 함수는 push() 입니다."
prompt = f'''
이전 답변 : {wrong_answer}
이 답변을 틀렸어 파이썬 문법에 맞지 않어
오류 원인을 분석(Reflection)하고 올바른 답을 수정해서 알려줘
'''
print(ask_llm(prompt))

In [ ]:
# 16 Multimodal Cot(멀티모달 Cot)
# 텍스트뿐만 아니라 이미지를 함께 보면서 단계별로 추론하는 것
client = OpenAI()
# 로컬파일은 직접 지정 X
uploaded = client.files.create(
    file = open(r'C:\LLM\openai\image.png', 'rb'),
    purpose='vision'
)
file_id = uploaded.id

reponse = client.chat.completions.create(
    model = 'gpt-5-nano',
    messages=[{
        'role':'user',
        'content':[
            {'type':'text', 'text' : '이 사진의 상황을 단계별로 추론해서 설명해줘'},
            {'type':'image_url', 
             'image_url':{'url':"https://github.com/pia222sk20/LLM2/raw/main/openai/image.png"} }
        ]
    }]
)
print(reponse.choices[0].message.content)

In [ ]:
# 17 Graph Prompting(그래프 프롬프팅)
# 테이터를 텍스트가 아니라 그래프(노드와 연결관계) 형태로 설명하여 관계추론 능력을 높임
prompt = '''
- (철수) -- 친구 -- (영희)
- (영희) -- 존경 -- (선생님)
- (선생님) -- 제자 -- (철수)

질문: 철수와 선생님의 관계를 그래프 구조를 보고 설명해줘 
'''
print(ask_llm(prompt))

In [ ]:
# 문서를 그래프구조 바꿔서 모델에 입력
# 문서->지식 그래프->그래프 프롬프트->LLM->RAG
prompt = '''
회사 지식그래프(Knowledge Graph)

(HR 부서) -- 관리 --> (채용프로세스)
(채용프로세스) -- 포함 --> (이력서검토)
(이력서검토) -- 검토자 --> (팀장)
(이력서검토) -- 검토자 --> (HR매니저)

사용자질문 : "채용과정에서 HR 부서는 어떤 역활을 하나?"

지식그래프 기반으로 답변해줘
'''
print(ask_llm(prompt))

In [ ]:
# 18 Meta-prompting(메타 프롬프팅)
# 하나의 거대한 문제를 해결하기 위해서 llm 스스로 하위 프롬프트를 여러개 생성하고 관리하는 Meta(최상의) 기법 - 오케스트라 지휘자 - 

# 문제 :  중소기업이 AI 도입 전략을 세울때 핵심 단계를 알려줘
# LLM 판단
    # 전략 전문가
    # 기술 전문가
    # 예산: ROI 전문가
# 3명이 필요하다고 하면 스스로 하위 프롬프트를 생성 --> 각각 실행 ->종합

# 전문가 3명을 정의하도록 llm에게 요청
meta_prompt = '''
우리는 다음 문제를 해결하려고 한다
중소기업이 AI 도입 전략을 세울때 핵심 단계를 알려줘

이 문제를 해결하기 위해 필요한 전문가 3명을 정의하고
각 전문가에 줄 개별 프롬프트를 만들어라

출력형식:
1. 전문가1 이름:
프롬프트:
2. 전문가2 이름:
프롬프트:
3. 전문가3 이름:
프롬프트:
'''
print(f'메타 프롬프트를 생성.....')
experts = ask_llm(meta_prompt)
print(experts)

In [134]:
experts

'1. 전문가1 이름: 중소기업 AI 전략 설계 전문가\n프롬프트:\n당신은 중소기업의 AI 도입 전략 설계 전문가다. 목표는 제한된 예산과 인력으로도 실행 가능한 “핵심 단계 로드맵”을 제시하는 것이다. 아래 요구사항을 반영해 8~12개의 핵심 단계로 구성된 로드맵을 만들어라.\n- 각 핵심 단계에 포함될 항목: 단계명, 목적, 추천 산출물(템플릿/문서), 선행 조건(데이터/기술/인력), 예상 소요 기간(주), 이전 단계와의 의존 관계\n- 우선순위 선정 프레임워크: 가치/비용/리스크 기반으로 사용 사례를 선별하는 방법 제시\n- 빠른 성과를 위한 Quick Wins 시나리오 포함\n- 데이터 준비, 기술 스택 선택, 거버넌스, 예산 및 ROI 추정 방법의 개략적 가이드 포함\n- 리스크 관리: 흔한 실패 요인 5가지와 이를 방지하는 대책 제시\n- 예시 적용: 제조업, 소매업, 서비스업 중 하나의 간단한 사례를 통해 맥락 설명\n- 출력 형식은 항목별 목록으로, 각 항목은 숫자 표기로 구성되도록 해줘\n- 필요 시 도식화 흐름(1-2문장)을 함께 포함해 전체 흐름을 이해시키게 해줘\n\n2. 전문가2 이름: 데이터 거버넌스 및 보안 전문가\n프롬프트:\n당신은 데이터 거버넌스와 보안 전문이다. 중소기업의 AI 도입에 필요한 데이터 측면의 핵심 체크리스트와 설계안을 제시하라.\n- 포함 내용: 데이터 거버넌스 프레임워크(데이터 소유 및 책임, 정책, 표준), 데이터 품질 관리 계획(데이터 품질 지표, 메타데이터/데이터 계보), 데이터 인프라 요구사항(데이터 레이크/웨어하우스, 접속 제어, 암호화, 백업), 데이터 보안(접근 제어, 인증/권한 관리, 모니터링, 사고 대응), 개인정보 보호 및 규정 준수(PII/민감정보 처리, 지역별 규정), 공급망 데이터 계약 및 리스크 관리(벤더 데이터 처리 계약), 데이터 파이프라인 설계의 실현 가능성 및 우선순위, 성공 지표(KPI)와 운영 체계.\n- 출력물 형식은 체크리스트, 도식화된 아키텍처 개요, 그리고 단계별 실

In [132]:
import re
exp_prompts = re.findall(r'프롬프트:\s*(.*)', experts)
# 각 전문가 llm 호출
result = [ ask_llm(prompt) for prompt in exp_prompts]

result

['다음은 제한된 예산과 인력으로도 실행 가능한 12단계의 핵심 로드맵입니다. 각 단계는 구체적 목표, 주요 활동, 기대 산출물, 권장 도구를 함께 제시해 바로 실행에 옮길 수 있도록 구성했습니다.\n\n1. 경영진 스폰서 확보 및 문제 정의\n- 목표: AI 도입의 목표를 명확히 하고 1~2개의 고임팩트 use case를 선정\n- 주요 활동: 경영진 워크숍 개최, 비즈니스 문제를 1~2문장으로 정의, 성공지표(KPI) 설정\n- 산출물: 우선순위 문제 목록, 선택된 MVP(use case 1개)와 KPI\n- 권장 도구/전략: 간단한 프레이밍 도구(RICE/ICE), 기존 회의록 활용\n\n2. 데이터 현황 진단 및 소유 체계 확립\n- 목표: 활용 가능한 데이터 원천과 책임자를 파악\n- 주요 활동: 데이터 인벤토리 작성, 데이터 소유자/접근 권한 파악, 개인정보/보안 이슈 목록화\n- 산출물: 데이터 소유자 맵, 데이터 소스 목록, 보안/프라이버시 기본 가이드\n- 권장 도구/전략: 스프레드시트 기반 데이터 카탈로그, 간단한 데이터 맵\n\n3. 데이터 품질 및 거버넌스 기본 구축\n- 목표: 데이터 품질 문제를 빠르게 해결할 기반 마련\n- 주요 활동: 기본 정제 규칙 수립(결측치 처리, 중복 제거, 포맷 통일), 품질 대시보드 시범 운영\n- 산출물: 데이터 품질 개선 로드맹, 샘플링된 품질 리포트\n- 권장 도구/전략: Python/pandas로 간단한 스크립트, 무료 ETL 도구로 시범 구성\n\n4. MVP 후보 use case 설계 및 선택\n- 목표: 1개 핵심 use case로 최소한의 기능을 구현하는 MVP 정의\n- 주요 활동: 데이터 준비 계획 확정, 모델링 접근법 결정(노코드/로우코드 vs 코딩), 평가 지표 확정\n- 산출물: MVP 설계 문서, 평가 지표, 성공 조건\n- 권장 도구/전략: 로우코드/노코드 도구 또는 파이썬 기반 간단 프로토타입\n\n5. 인프라 및 도구 선택\n- 목표: 비용 효율적이고 확장 가능한 도구 조합 

In [133]:
final_prompt = f'''
다음은 3명의 전문가가 낸 의견이야

1.{result[0]}
2.{result[1]}
3.{result[2]}

위 내용을 종합해서 
중소기업이 AI 도입 전략을 세울때 핵심 단계를 정리해서 작성해
단계적으로, 명확하게, 문장이나 문맥에 어색함 없이, 한글로 작성해
'''
print(ask_llm(final_prompt))

# 1. Meta LLM
# 2. Sub  LLM
# 3. Aggregator(종합 단계)

다음은 중소기업이 AI 도입 전략을 체계적으로 수립할 때 참고할 핵심 단계들입니다. 각 단계는 목적, 주요 활동, 산출물, 권장 도구/전략으로 구성되어 실행에 바로 옮길 수 있도록 정리했습니다.

1) 경영진 스폰서 확보 및 문제 정의
- 목표: AI 도입의 목적을 명확히 하고 1~2개의 고임팩트 use case를 선정
- 주요 활동: 경영진 워크숍 개최, 비즈니스 문제를 1~2문장으로 정의, 성공지표(KPI) 설정
- 산출물: 우선순위 문제 목록, 선택된 MVP(use case 1개)와 KPI
- 권장 도구/전략: 간단한 프레이밍 도구(RICE/ICE), 기존 회의록 활용

2) 데이터 현황 진단 및 소유 체계 확립
- 목표: 활용 가능한 데이터 원천과 책임자를 파악
- 주요 활동: 데이터 인벤토리 작성, 데이터 소유자/접근 권한 파악, 개인정보/보안 이슈 목록화
- 산출물: 데이터 소유자 맵, 데이터 소스 목록, 보안/프라이버시 기본 가이드
- 권장 도구/전략: 스프레드시트 기반 데이터 카탈로그, 간단한 데이터 맵

3) 데이터 품질 및 거버넌스 기본 구축
- 목표: 데이터 품질 문제를 빠르게 해결할 기반 마련
- 주요 활동: 기본 정제 규칙 수립(결측치 처리, 중복 제거, 포맷 통일), 품질 대시보드 시범 운영
- 산출물: 데이터 품질 개선 로드맹, 샘플링된 품질 리포트
- 권장 도구/전략: Python(pandas)으로 간단 스크립트 작성, 무료 ETL 도구로 시범 구성

4) MVP 후보 use case 설계 및 선택
- 목표: 1개 핵심 use case로 최소한의 기능을 구현하는 MVP 정의
- 주요 활동: 데이터 준비 계획 확정, 모델링 접근법 결정(노코드/로우코드 vs 코딩), 평가 지표 확정
- 산출물: MVP 설계 문서, 평가 지표, 성공 조건
- 권장 도구/전략: 로우코드/노코드 도구 또는 Python으로 간단 프로토타입

5) 인프라 및 도구 선택
- 목표: 비용 효율적이고 확장 가능한 도구 조합 선택
- 주요 활동: 클라우드 무료 티어 